In [1]:
from models.DistMult import DistMult_Lite
from models.Complex import Complex
from models.ConvE import ConvE, ConvE_args

from utils.loaders import load_data, get_onehots
from utils.evaluation_metrics import SRR, auprc_auroc_ap

import torch
import numpy as np
from sklearn.utils import shuffle

In [8]:
from tqdm import tqdm

In [9]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary, Edge_list, Edge_features, Drug_graph_list, Protein_graph_list = get_files()
entities = int(len(lookup)/2)

In [3]:
# Drug_list = list(set(data[:,0]))
# Protein_list = list(set(data[:,2]))

# Drug_graph_dict = {x : y for x, y in zip(Drug_list, Drug_graph_list)}
# Protein_graph_dict = {x : y for x, y in zip(Protein_list, Protein_graph_list)}

# filtered_data = [x for x in data if not isinstance(Drug_graph_dict[x[0]], str)] 
# filtered_data = [x for x in filtered_data if not isinstance(Protein_graph_dict[x[2]], str)] 

# data = filtered_data

In [12]:
protien_ids = list(set(data[:,2]))
protien_ids = torch.LongTensor(protien_ids)

In [14]:
number_of_batches = 5
number_of_epochs = 20
x = shuffle(data)
test_data = x[:50]

In [15]:
model = DistMult_Lite(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(number_of_epochs):


        #training stage 
        model.train()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            # print(obj.shape)
            rel = torch.LongTensor(relationships[index])
            # print(len(rel))
            
            preprocessed_target = torch.squeeze(torch.LongTensor(subjects[index]))
            target = torch.squeeze(torch.stack([(protien_ids == x).nonzero() for x in preprocessed_target]))
            # print(len(subj))


            optimiser.zero_grad()
            pred = model.forward(obj = obj, rel = rel, subj = protien_ids)
            
            loss = model.loss(pred, target)
            
            loss.backward()
            optimiser.step()


        #evaluation stage
        model.eval()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)
        total_sum_reciporical_rank = torch.zeros(1)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            
            preprocessed_target = torch.squeeze(torch.LongTensor(subjects[index]))
            target = torch.squeeze(torch.stack([(protien_ids == x).nonzero() for x in preprocessed_target]),1)

            predictions = model.forward(obj = obj, rel = rel, subj = protien_ids)
            srr = SRR(predictions, target) 
            # print(predictions.shape)
            # print(targets.shape)
            total_sum_reciporical_rank = total_sum_reciporical_rank + srr

        print('mean reciporical rank is...')
        print(total_sum_reciporical_rank / len(test_data))


mean reciporical rank is...
tensor([0.0074])
mean reciporical rank is...
tensor([0.0122])
mean reciporical rank is...
tensor([0.0207])
mean reciporical rank is...
tensor([0.0503])
mean reciporical rank is...
tensor([0.0976])
mean reciporical rank is...
tensor([0.1795])
mean reciporical rank is...
tensor([0.3067])
mean reciporical rank is...
tensor([0.4042])
mean reciporical rank is...
tensor([0.5259])
mean reciporical rank is...
tensor([0.6396])
mean reciporical rank is...
tensor([0.7506])
mean reciporical rank is...
tensor([0.8354])
mean reciporical rank is...
tensor([0.9126])
mean reciporical rank is...
tensor([0.9617])
mean reciporical rank is...
tensor([0.9635])
mean reciporical rank is...
tensor([0.9803])
mean reciporical rank is...
tensor([0.9808])
mean reciporical rank is...
tensor([0.9817])
mean reciporical rank is...
tensor([0.9900])
mean reciporical rank is...
tensor([1.])


In [ ]:
def get_adj_mask(max_nodes, graph):
    num_nodes = graph.num_nodes
    mask = np.zeros([max_nodes,max_nodes], dtype = bool)
    mask[0:num_nodes][0:num_nodes] = True
    
    adjacency = np.zeros([max_nodes,max_nodes]) # Check if Dtype int is needed! 
    edges = graph.edge_index.T
    for edge in edges:
        adjacency[edge[0]][edge[1]] = 1
        adjacency[edge[1]][edge[0]] = 1
    return Data(x = store.x, adj = adjacency, mask = mask)
    

max_graph_size = 2000
graphs_with_masks = []
for store, PDB_key in tqdm(zip(stored_graphs, PDB_keys)):
    try:
        if store.num_nodes > max_graph_size:
            # print(store, ' too big')
            graphs_with_masks.append('{} too big'.format(PDB_key))
        else:
            graphs_with_masks.append(get_adj_mask(max_graph_size, store))
    except:
        # print(PDB_key, ' missing')
        graphs_with_masks.append('{} missing'.format(PDB_key))